# Import Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

ModuleNotFoundError: No module named 'folium'

# Download and Explore Dataset

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


# Tranform the data into a pandas dataframe

In [3]:
neighborhoods_data = newyork_data['features']

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 


neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


# Use geopy library 

In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


# Slash  dataframe to include only Manhattan area

In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


# Create map of New York city 

In [8]:
import folium

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Use of Foursquare API

In [7]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
        

            
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

          
            response = requests.get(url).json()
            results = response["response"]['venues']

           
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [8]:
LIMIT = 500
radius = 5000 
CLIENT_ID = '5JGDTGXHE55D4RQETWKJP34MJKUJ4TOXV4A3VBK0DZXFX1Y5' 
CLIENT_SECRET = 'VPRR2KODGEMLGIOIB4E5GPXC2VFAYBJUTW1CJPMBTNICQEAG' 
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5JGDTGXHE55D4RQETWKJP34MJKUJ4TOXV4A3VBK0DZXFX1Y5
CLIENT_SECRET:VPRR2KODGEMLGIOIB4E5GPXC2VFAYBJUTW1CJPMBTNICQEAG


In [9]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_greek = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d10e941735')
newyork_venues_greek.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Greek Express,40.883703,-73.904788,Greek Restaurant
1,Chinatown,40.715618,-73.994279,Kiki's,40.714476,-73.992036,Greek Restaurant
2,Chinatown,40.715618,-73.994279,Kiki’s Grill & Rotisserie,40.714303,-73.991796,Greek Restaurant
3,Chinatown,40.715618,-73.994279,GFG Bakery and Café,40.710254,-74.005620,Bakery
4,Chinatown,40.715618,-73.994279,Greek Shack,40.715501,-73.993529,Diner


In [10]:
newyork_venues_greek.shape

(467, 7)

In [12]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [13]:
map_newyork_greek = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_greek, 'red', map_newyork_greek)

map_newyork_greek

In [14]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [15]:
manhattan_grouped = newyork_venues_greek.groupby('Neighborhood').count()
manhattan_grouped


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,6,6,6,6,6,6
Carnegie Hill,4,4,4,4,4,4
Central Harlem,2,2,2,2,2,2
Chelsea,12,12,12,12,12,12
Chinatown,9,9,9,9,9,9
Civic Center,12,12,12,12,12,12
Clinton,24,24,24,24,24,24
East Harlem,2,2,2,2,2,2
East Village,15,15,15,15,15,15


# Neighborhood analysis

In [16]:
manhattan_onehot = pd.get_dummies(newyork_venues_greek[['Venue Category']], prefix="", prefix_sep="")


manhattan_onehot['Neighborhood'] = newyork_venues_greek['Neighborhood'] 


fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Bakery,Diner,Falafel Restaurant,Food Truck,Greek Restaurant,Mediterranean Restaurant,Souvlaki Shop,Street Food Gathering,Wine Bar
0,Marble Hill,0,0,0,0,0,1,0,0,0,0
1,Chinatown,0,0,0,0,0,1,0,0,0,0
2,Chinatown,0,0,0,0,0,1,0,0,0,0
3,Chinatown,0,1,0,0,0,0,0,0,0,0
4,Chinatown,0,0,1,0,0,0,0,0,0,0


In [17]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Bakery,Diner,Falafel Restaurant,Food Truck,Greek Restaurant,Mediterranean Restaurant,Souvlaki Shop,Street Food Gathering,Wine Bar
0,Battery Park City,0.000000,0.333333,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.000000,0.000000,0.083333,0.833333,0.000000,0.000000,0.000000,0.083333
4,Chinatown,0.000000,0.111111,0.111111,0.000000,0.000000,0.777778,0.000000,0.000000,0.000000,0.000000
5,Civic Center,0.000000,0.166667,0.083333,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000
6,Clinton,0.041667,0.000000,0.000000,0.041667,0.041667,0.791667,0.000000,0.000000,0.041667,0.041667
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.000000,0.000000,0.000000,0.066667,0.800000,0.066667,0.000000,0.000000,0.066667
9,Financial District,0.000000,0.333333,0.000000,0.000000,0.166667,0.500000,0.000000,0.000000,0.000000,0.000000


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Greek Restaurant,Bakery,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,American Restaurant
1,Carnegie Hill,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
2,Central Harlem,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
3,Chelsea,Greek Restaurant,Wine Bar,Food Truck,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Falafel Restaurant,Diner,Bakery,American Restaurant
4,Chinatown,Greek Restaurant,Diner,Bakery,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,American Restaurant


In [31]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)


kmeans.labels_[0:10]

array([3, 0, 0, 2, 4, 4, 2, 0, 2, 3], dtype=int32)

In [21]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0.0,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,4.0,Greek Restaurant,Diner,Bakery,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,American Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0.0,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0.0,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant


In [22]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
2,Washington Heights,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
4,Hamilton Heights,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
5,Manhattanville,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
6,Central Harlem,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
7,East Harlem,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
8,Upper East Side,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
9,Yorkville,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
10,Lenox Hill,Greek Restaurant,Mediterranean Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
11,Roosevelt Island,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant


In [23]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Lower East Side,Greek Restaurant,Wine Bar,Diner,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Bakery,American Restaurant
31,Noho,Greek Restaurant,Wine Bar,Food Truck,Diner,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Falafel Restaurant,Bakery,American Restaurant
37,Stuyvesant Town,Greek Restaurant,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant


In [24]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Greek Restaurant,Souvlaki Shop,Wine Bar,Street Food Gathering,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,Bakery,American Restaurant
14,Clinton,Greek Restaurant,Wine Bar,Street Food Gathering,Food Truck,Falafel Restaurant,American Restaurant,Souvlaki Shop,Mediterranean Restaurant,Diner,Bakery
15,Midtown,Greek Restaurant,Food Truck,Street Food Gathering,Mediterranean Restaurant,Falafel Restaurant,Wine Bar,Souvlaki Shop,Diner,Bakery,American Restaurant
16,Murray Hill,Greek Restaurant,Food Truck,Street Food Gathering,Mediterranean Restaurant,Falafel Restaurant,Wine Bar,Souvlaki Shop,Diner,Bakery,American Restaurant
17,Chelsea,Greek Restaurant,Wine Bar,Food Truck,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Falafel Restaurant,Diner,Bakery,American Restaurant
18,Greenwich Village,Greek Restaurant,Food Truck,Bakery,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Falafel Restaurant,Diner,American Restaurant
19,East Village,Greek Restaurant,Wine Bar,Mediterranean Restaurant,Food Truck,Street Food Gathering,Souvlaki Shop,Falafel Restaurant,Diner,Bakery,American Restaurant
33,Midtown South,Greek Restaurant,Street Food Gathering,Food Truck,Falafel Restaurant,Wine Bar,Souvlaki Shop,Mediterranean Restaurant,Diner,Bakery,American Restaurant
34,Sutton Place,Greek Restaurant,Food Truck,Souvlaki Shop,Mediterranean Restaurant,Wine Bar,Street Food Gathering,Falafel Restaurant,Diner,Bakery,American Restaurant
35,Turtle Bay,Greek Restaurant,Food Truck,Souvlaki Shop,Mediterranean Restaurant,Wine Bar,Street Food Gathering,Falafel Restaurant,Diner,Bakery,American Restaurant


In [25]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Battery Park City,Greek Restaurant,Bakery,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,Diner,American Restaurant
29,Financial District,Greek Restaurant,Bakery,Food Truck,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Falafel Restaurant,Diner,American Restaurant


In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Greek Restaurant,Diner,Bakery,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,American Restaurant
22,Little Italy,Greek Restaurant,Bakery,Food Truck,Diner,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Falafel Restaurant,American Restaurant
23,Soho,Greek Restaurant,Food Truck,Diner,Bakery,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Falafel Restaurant,American Restaurant
32,Civic Center,Greek Restaurant,Bakery,Diner,Wine Bar,Street Food Gathering,Souvlaki Shop,Mediterranean Restaurant,Food Truck,Falafel Restaurant,American Restaurant


In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters